In [1]:
!pip3 install flask_cors
!pip3 install pyngrok
!pip3 install openpyxl

In [1]:
import pandas as pd
import random

def getRoom(room, floor):
    # Load the CSV file into a DataFrame
    df = pd.read_csv('roomData.csv')
    my_list = ["running", "down", "service"]
    
    payload = {
        "room":room,
        "floor":floor,
        "roomService":{
            "ElectricalPanel":random.choice(my_list),
            "PlumbingSystem":random.choice(my_list),
            "FireAlarm":random.choice(my_list),
            "Elevator":random.choice(my_list),
            "HVAC":random.choice(my_list)
        }
    }
    return payload

def getNotify():
    anomaly = pd.read_excel('notif_anomalies.xlsx')
    # Initialize an empty list to store notification data
    notifications = []

    # Iterate through rows in the DataFrame
    for index, row in anomaly.iterrows():
        asset_type = row['AssetType']
        room = str(row['Room'])  # Assuming 'Room' is a numeric field in the Excel file
        floor = str(row['Floor'])  # Assuming 'Floor' is a numeric field in the Excel file
    
        # Create a notification dictionary for the current row
        notification = {
            "AssetType": asset_type,
            "room": room,  # Ensure room and floor are 3 digits with leading zeros
            "floor": floor
        }
    
        # Append the notification to the list
        notifications.append(notification)
        
    payload = {"notifications": notifications}
    return payload

def getFutureData():
    df = pd.read_excel("test_excel.xlsx")
    # List of possible AssetTypes
    asset_types = ["Elevator", "Fire Alarm", "Electrical Panel", "Plumbing System", "HVAC"]
    # Initialize an empty list to store future service data
    future_services = []

    # Iterate through rows in the DataFrame
    for index, row in df.iterrows():
        asset_type = random.choice(asset_types)  # Randomly select an AssetType
        latest_service_date = row['LatestServiceDate']
        avg_service_time = pd.Timedelta(days=row['Averageservicetimes'])
        delay_time = pd.Timedelta(days=row['DelayTime'])
    
        service_required_from = latest_service_date + avg_service_time
        service_before = service_required_from + delay_time
        room = str(row['Room'])  # Ensure room is 3 digits with leading zeros

        # Create a future service dictionary for the current row
        future_service = {
            "AssetType": asset_type,
            "ServiceRequiredFrom": service_required_from.strftime("%m/%d/%Y"),
            "ServiceBefore": service_before.strftime("%m/%d/%Y"),
            "room": room
        }

        # Append the future service to the list
        future_services.append(future_service)

    # Create the final payload JSON
    payload = {"futureServices": future_services}

    return payload


from flask import Flask, jsonify, make_response, render_template, request
from flask_cors import CORS
import requests
from pyngrok import ngrok

port_no = 5000
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

# Set up ngrok
ngrok.set_auth_token("2VVfjzl2D7XpYu5bLLGCdEhBUcZ_5LFBapVYiLcefZNP5vFWj")
public_url = ngrok.connect(port_no, bind_tls=True)
print(f"To access the Global link, please click {public_url}")

@app.route('/getNotification', methods=['POST'])
def notificationData():
    payload = {
        "notifications":[{
	        "AssetType":"Name",
            "room":"000",
            "floor":"000"}]
        }
    return jsonify(getNotify())

@app.route('/getFutureSeries', methods=['POST'])
def futureData():
    payload = {
        "futureServices":[{
	        "AssetType":"Name",
            "ServiceRequiredFrom": "MM/DD/YYYY",
            "ServiceBefore": "MM/DD/YYYY",
            "room":"000"
        }]
    }
    return jsonify(getFutureData()) 

@app.route('/getRoomData', methods=['POST'])
def roomData():
    data = request.get_json()
    print(data)
    response = getRoom(data['room'], data['floor'])
    # payload = {
    #     "room":"roomNumber",
    #     "floor":"floor",
    #     "roomService":{
    #         "ElectricalPanel":"running",
    #         "PlumbingSystem":"down",
    #         "FireAlarm":"service",
    #         "Elevator":"service",
    #         "HVAC":"running"
    #     }
    # }
    return jsonify(response) 


print(f"To access the Gloable link please click {public_url}")

# Define a route to serve static files
app.static_folder = 'static'

@app.route('/')
def home():
    return render_template("./index.html")

@app.route('/dashboard')
def dashboard():
    return render_template("./dashboard.html")

app.run(port=port_no)

t=2023-09-17T14:53:58+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/home/codespace/.config/ngrok/ngrok.yml legacy_path=/home/codespace/.ngrok2/ngrok.yml


To access the Global link, please click NgrokTunnel: "https://2b7e-20-106-101-232.ngrok-free.app" -> "http://localhost:5000"
To access the Gloable link please click NgrokTunnel: "https://2b7e-20-106-101-232.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Sep/2023 14:54:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2023 14:54:01] "GET /static/main.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:54:01] "GET /static/server.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:54:01] "GET /static/notifications.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:54:12] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2023 14:54:13] "GET /static/notifications.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:54:13] "GET /static/server.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:54:13] "GET /static/main.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:54:19] "POST /getRoomData HTTP/1.1" 200 -


{'floor': '3', 'room': '105'}


127.0.0.1 - - [17/Sep/2023 14:55:07] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2023 14:55:07] "GET /static/server.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:55:07] "GET /static/main.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:55:07] "GET /static/notifications.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:55:12] "POST /getRoomData HTTP/1.1" 200 -


...

{'floor': '4', 'room': '107'}


127.0.0.1 - - [17/Sep/2023 14:56:32] "POST /getRoomData HTTP/1.1" 200 -


{'floor': '3', 'room': '102'}


127.0.0.1 - - [17/Sep/2023 14:57:11] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2023 14:57:11] "GET /static/notifications.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:57:11] "GET /static/main.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:57:11] "GET /static/server.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2023 14:57:25] "POST /getRoomData HTTP/1.1" 200 -


{'floor': '4', 'room': '104'}
